In [245]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
%matplotlib inline
import xarray

In [250]:
# TEST
batch_path = '/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/'

In [251]:
# TEST
reference = pd.read_csv(batch_path + os.listdir(batch_path)[0], sep="\\t", engine='python')

In [253]:
# TEST
sample_df = pd.DataFrame()
# create empty dataframe

num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
# how many files there are in this datagrame

# Following for loop appends all of the files except the first one into 1 big dataframe
for f in range(num_files-1):
    standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
    sample_df = sample_df.append(standard)
sorted_df = sample_df.sort_values(by=['RT (min)'])

sorted_df

,PeakID,Title,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,...,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum
0,0,Unknown,91,0.506767,0.539950,0.639567,233.0005,4060.826416,20754.892578,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,22.221230,233.00046:4072 233.04866:1239 234.00003:357 23...,NaN
0,0,Unknown,95,0.540933,0.563067,0.635017,158.0128,9714.203125,46207.648438,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,11.739810,158.01242:9701 158.12152:305 158.99962:2944 16...,NaN
0,0,Unknown,95,0.538633,0.566300,0.638233,158.0130,10269.208984,50936.386719,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,11.337680,158.0128:10488 158.12692:270 159.00041:2848 15...,NaN
0,0,Unknown,96,0.535083,0.568283,0.634683,119.0805,8126.946777,34228.234375,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,18.259390,119.08052:9467 120.02747:206 120.08226:405 120...,NaN
0,0,Unknown,96,0.538983,0.572200,0.633067,158.0118,10673.823242,51182.355469,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,15.399140,158.01176:11214 158.99854:3795 159.99057:217 1...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1794,Unknown,3950,21.859020,21.897750,21.925420,165.0152,3064.297852,9484.580078,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,18.259980,164.9718:252 165.02225:3562 166.0072:348 166.0...,NaN
30,30,Unknown,3974,21.944930,22.005750,22.033380,721.5085,94663.734375,495353.656250,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,0.826930,721.50848:95321 722.51111:45311 722.74585:1237...,NaN
31,31,Unknown,4004,22.149470,22.171570,22.188150,721.5081,93614.242188,216024.078125,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,0.365460,721.50806:94704 722.5116:44038 722.81744:619 7...,NaN
208,208,Unknown,4012,22.149850,22.238380,22.282650,232.9997,5672.978516,41947.949219,NaN,...,False,False,NaN,NaN,NaN,NaN,NaN,9.743987,232.99973:6238 233.13161:255 234.0027:413,NaN


In [264]:
peak_num = 0
listing = []
for x in range(len(sorted_df)):
    if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)):
        listing.append(peak_num)
    else:
        peak_num = peak_num + 1
        listing.append(peak_num)
sorted_df['unique peak'] = listing

In [262]:
x=1
(abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)

False

In [265]:
sorted_df

,PeakID,Title,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,...,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum,unique peak
0,0,Unknown,91,0.506767,0.539950,0.639567,233.0005,4060.826416,20754.892578,NaN,...,False,NaN,NaN,NaN,NaN,NaN,22.221230,233.00046:4072 233.04866:1239 234.00003:357 23...,NaN,1
0,0,Unknown,95,0.540933,0.563067,0.635017,158.0128,9714.203125,46207.648438,NaN,...,False,NaN,NaN,NaN,NaN,NaN,11.739810,158.01242:9701 158.12152:305 158.99962:2944 16...,NaN,2
0,0,Unknown,95,0.538633,0.566300,0.638233,158.0130,10269.208984,50936.386719,NaN,...,False,NaN,NaN,NaN,NaN,NaN,11.337680,158.0128:10488 158.12692:270 159.00041:2848 15...,NaN,2
0,0,Unknown,96,0.535083,0.568283,0.634683,119.0805,8126.946777,34228.234375,NaN,...,False,NaN,NaN,NaN,NaN,NaN,18.259390,119.08052:9467 120.02747:206 120.08226:405 120...,NaN,3
0,0,Unknown,96,0.538983,0.572200,0.633067,158.0118,10673.823242,51182.355469,NaN,...,False,NaN,NaN,NaN,NaN,NaN,15.399140,158.01176:11214 158.99854:3795 159.99057:217 1...,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1794,Unknown,3950,21.859020,21.897750,21.925420,165.0152,3064.297852,9484.580078,NaN,...,False,NaN,NaN,NaN,NaN,NaN,18.259980,164.9718:252 165.02225:3562 166.0072:348 166.0...,NaN,234375
30,30,Unknown,3974,21.944930,22.005750,22.033380,721.5085,94663.734375,495353.656250,NaN,...,False,NaN,NaN,NaN,NaN,NaN,0.826930,721.50848:95321 722.51111:45311 722.74585:1237...,NaN,234376
31,31,Unknown,4004,22.149470,22.171570,22.188150,721.5081,93614.242188,216024.078125,NaN,...,False,NaN,NaN,NaN,NaN,NaN,0.365460,721.50806:94704 722.5116:44038 722.81744:619 7...,NaN,234377
208,208,Unknown,4012,22.149850,22.238380,22.282650,232.9997,5672.978516,41947.949219,NaN,...,False,NaN,NaN,NaN,NaN,NaN,9.743987,232.99973:6238 233.13161:255 234.0027:413,NaN,234378


In [252]:
# TEST: This is the function but I just wrote comments on all of the stuff
sample_df = pd.DataFrame()
# create empty dataframe

num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
# how many files there are in this datagrame

# Following for loop appends all of the files except the first one into 1 big dataframe
for f in range(num_files-1):
    standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
    sample_df = sample_df.append(standard)
sorted_df = sample_df.sort_values(by=['RT (min)'])  # This is the problem line
# so lets say R1 = 100, m/z1 = 30 R2 = 100.1,  m/z 2= 5000, R3 = 100.2  m/z3 = 30
# because i only compare them to the previous row
# R1 and R3 should be appended but  they arent

#Following part will create a new column called unique peaks 
peak_num = 0
listing = []
for x in range(len(sorted_df)):
    if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) >= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) >= 0.002)):
        peak_num = peak_num + 1
        listing.append(peak_num)
    else:
        listing.append(peak_num)
sorted_df['unique peak'] = listing
# sorted_df  

# groups the rows by unique peaks and takes an average of the peaks
sorted_df.groupby(['unique peak']).mean()

,PeakID,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,Reference RT,...,RT matched,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MSMS spectrum
unique peak,,,,,,,,,,,,,,,,,,,,,
1,153.743527,112.740483,0.535378,0.661481,0.779141,412.583322,8964.371273,46573.708523,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,92.592653,NaN
2,196.870130,328.123377,1.691870,1.853062,1.999559,234.350985,28314.047091,607912.586665,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,233.174245,NaN
3,137.040000,392.080000,2.087153,2.206684,2.329526,374.179368,18046.202769,222061.908320,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,49.053393,NaN
4,264.006061,445.769697,2.373490,2.504253,2.721610,174.782970,11073.977698,142147.638308,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,76.864790,NaN
5,1352.473490,2028.237806,11.114872,11.261277,11.441698,466.499840,37662.496166,423389.421066,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,281.263114,NaN
6,1941.116057,3799.061675,20.924411,21.060561,21.216751,278.471190,11269.373423,88086.206284,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,82.297124,NaN
7,2124.250000,3890.750000,21.530592,21.569333,21.617748,322.511225,3036.459167,8445.305786,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,6.929345,NaN
8,2133.688525,3920.426230,21.553757,21.732378,21.779460,178.116756,18544.202381,161566.596440,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,59.848713,NaN
9,1405.250000,3945.000000,21.830605,21.867955,21.914987,160.494450,13376.078491,70737.688965,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,18.841287,NaN


In [266]:
def sort_peaks(batch_path):
    sample_df = pd.DataFrame()
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for f in range(num_files-1):
        standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
        sample_df = sample_df.append(standard)
    
    sorted_df = sample_df.sort_values(by=['RT (min)'])
    
    peak_num = 0
    listing = []
    for x in range(len(sorted_df)):
        if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)):
            listing.append(peak_num)
        else:
            peak_num = peak_num + 1
            listing.append(peak_num)
    sorted_df['unique peak'] = listing
    return sorted_df.groupby(['unique peak']).mean()

In [267]:
%time sort_peaks('/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/')

CPU times: user 13.9 s, sys: 1.06 s, total: 15 s
Wall time: 15.3 s


,PeakID,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,Reference RT,...,RT matched,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MSMS spectrum
unique peak,,,,,,,,,,,,,,,,,,,,,
1,0.0,91.0,0.506767,0.539950,0.639567,233.0005,4060.826416,20754.892578,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,22.221230,NaN
2,0.0,95.0,0.539783,0.564683,0.636625,158.0129,9991.706055,48572.017578,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,11.538745,NaN
3,0.0,96.0,0.535083,0.568283,0.634683,119.0805,8126.946777,34228.234375,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,18.259390,NaN
4,0.0,96.0,0.538983,0.572200,0.633067,158.0118,10673.823242,51182.355469,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,15.399140,NaN
5,0.0,97.0,0.536383,0.575133,0.647083,117.1130,6957.079590,22054.017578,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,19.946600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234375,1794.0,3950.0,21.859020,21.897750,21.925420,165.0152,3064.297852,9484.580078,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,18.259980,NaN
234376,30.0,3974.0,21.944930,22.005750,22.033380,721.5085,94663.734375,495353.656250,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.826930,NaN
234377,31.0,4004.0,22.149470,22.171570,22.188150,721.5081,93614.242188,216024.078125,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.365460,NaN


In [268]:
# Sorted by Precursor m/z
def sort_peaks2(batch_path):
    sample_df = pd.DataFrame()
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for f in range(num_files-1):
        standard = pd.read_csv(batch_path + os.listdir(batch_path)[f+1], sep="\\t", engine='python')
        sample_df = sample_df.append(standard)
    
    sorted_df = sample_df.sort_values(by=['Precursor m/z'])
    
    peak_num = 0
    listing = []
    for x in range(len(sorted_df)):
        if ((abs(sorted_df.iat[x,4]-sorted_df.iat[(x-1),4]) <= 0.05) & (abs(sorted_df.iat[x,6]-sorted_df.iat[(x-1),6]) <= 0.002)):
            listing.append(peak_num)
        else:
            peak_num = peak_num + 1
            listing.append(peak_num)
    sorted_df['unique peak'] = listing
    return sorted_df.groupby(['unique peak']).mean()

In [271]:
j = sort_peaks2('/mnt/c/Users/nozom/Desktop/mass-suite/example_data/batch/')

j.sort_values(by=['RT (min)'])

,PeakID,Scans,RT left(min),RT (min),RT right (min),Precursor m/z,Height,Area,Model masses,Reference RT,...,RT matched,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MSMS spectrum
unique peak,,,,,,,,,,,,,,,,,,,,,
12915,0.0,91.0,0.506767,0.539950,0.639567,233.0005,4060.826416,20754.892578,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,22.221230,NaN
3994,0.0,95.0,0.540933,0.563067,0.635017,158.0128,9714.203125,46207.648438,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,11.739810,NaN
4029,0.0,95.0,0.538633,0.566300,0.638233,158.0130,10269.208984,50936.386719,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,11.337680,NaN
1226,0.0,96.0,0.535083,0.568283,0.634683,119.0805,8126.946777,34228.234375,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,18.259390,NaN
3884,0.0,96.0,0.538983,0.572200,0.633067,158.0118,10673.823242,51182.355469,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,15.399140,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,1794.0,3950.0,21.859020,21.897750,21.925420,165.0152,3064.297852,9484.580078,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,18.259980,NaN
67482,30.0,3974.0,21.944930,22.005750,22.033380,721.5085,94663.734375,495353.656250,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.826930,NaN
67412,31.0,4004.0,22.149470,22.171570,22.188150,721.5081,93614.242188,216024.078125,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.365460,NaN
